In [ ]:
''' This test script builds and runs three predictive models. The model with the lowest MSE score is
    used to show the most important features'''

__author__ = 'skyler bullard'

__copyright__ = 'Copyright 2018, Data Science Dream Job LLC'

In [2]:
#import everything we might need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

In [ ]:
path = 'C:/Users/sprtn/jupyter_project_files/salarypredictionportfolio/DSDJ-data-SalaryPredictions/data/'

test_features = pd.read_csv( open(path + 'test_features.csv') )
train_features = pd.read_csv( open(path + 'train_features.csv') )
train_salaries = pd.read_csv( open(path + 'train_salaries.csv') )

In [ ]:
#define variables
categorical_vars = [ 'companyId' , 'jobType' , 'degree' , 'major' , 'industry' ]
numerical_vars = [ 'yearsExperience' , 'milesFromMetropolis' ]
target_var = 'salary'

raw_